Домашнее задание 09

    - Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
    - Сделайте предсказания на всем df_val. Посчитайте метрику качества.
    - Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.2 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 60.5 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 8.2 MB/s 


In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from transformers import pipeline
from tqdm import tqdm
from torchmetrics import F1Score

cointegrated/rubert-tiny-toxicity

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification

# load tokenizer and model weights
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny-toxicity')
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny-toxicity')

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/377 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.0M [00:00<?, ?B/s]

In [5]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, element

In [6]:
model.classifier

Linear(in_features=312, out_features=5, bias=True)

In [8]:
# Проверка работы модели

sentiment = pipeline("text-classification", model='cointegrated/rubert-tiny-toxicity')
sentiment("Задание выполнено отлично!")

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

[{'label': 'non-toxic', 'score': 0.999870777130127}]

In [9]:
# Проверка работы токенайзера

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=15, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[    2,  3086, 10885, 22723,   871, 24302,  3464, 10880,     3,     0,
             0,     0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])


In [14]:
!unzip drive/MyDrive/data.zip

Archive:  drive/MyDrive/data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [15]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [16]:
df_train.head(5)

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [17]:
idx = np.random.randint(1,1000)
print(f'Индекс: {idx}')
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

Индекс: 74
Тяжелый выбор между парикхмахерской и кино)или и то и другое)
label is 1


[{'label': 'non-toxic', 'score': 0.9998155236244202}]

In [18]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

Датасет и даталоадер

In [20]:
class ToxDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny-toxicity')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [21]:
y_train = df_train['class']
y_val = df_val['class']

train_dataset = ToxDataset(df_train['text'], y_train)
valid_dataset = ToxDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=0)

In [22]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([128, 1, 10])


In [25]:
#класс модели
class BertForSequenceClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.pretrained_model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny-toxicity')
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        pooled_output = self.pretrained_model(input_ids=x, attention_mask=mask,return_dict=False)[0]  #(B, 2)
        out = self.sigm(pooled_output)
        return out

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [27]:
model = BertForSequenceClassifier().to(device)

In [28]:
model.pretrained_model.classifier

Linear(in_features=312, out_features=5, bias=True)

In [29]:
valid_f1 = F1Score().to(device)

total_acc_val = 0
for val_input, val_label in valid_loader:
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    valid_f1(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
    
print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {valid_f1.compute().item():.3f}')

Val accuracy: 0.471, F1 score: 0.471



Дообучаем модель на тренировочных данных и проверка результата на валидационных данных

In [30]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = Adam(model.pretrained_model.parameters(), lr=0.001)  

In [38]:
train_f1 = F1Score().to(device)
valid_f1 = F1Score().to(device)

for epoch_num in range(7):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)
        
        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        train_f1(output, train_label)

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

        valid_f1(output, val_label)

    # выведем метрики      
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f} \
        | Val f1: {valid_f1.compute().item(): .3f}')
    
    train_f1.reset()
    valid_f1.reset()


100%|██████████| 1418/1418 [00:27<00:00, 52.49it/s]


Epochs: 1 | Train Loss:  0.009         | Train Accuracy:  0.495         | Train f1:  0.495         | Val Loss:  0.009         | Val Accuracy:  0.495         | Val f1:  0.495



100%|██████████| 1418/1418 [00:27<00:00, 52.21it/s]


Epochs: 2 | Train Loss:  0.009         | Train Accuracy:  0.494         | Train f1:  0.494         | Val Loss:  0.009         | Val Accuracy:  0.495         | Val f1:  0.495



100%|██████████| 1418/1418 [00:28<00:00, 50.46it/s]


...

Epochs: 5 | Train Loss:  0.009         | Train Accuracy:  0.493         | Train f1:  0.493         | Val Loss:  0.009         | Val Accuracy:  0.495         | Val f1:  0.495



100%|██████████| 1418/1418 [00:27<00:00, 51.26it/s]


Epochs: 6 | Train Loss:  0.009         | Train Accuracy:  0.493         | Train f1:  0.493         | Val Loss:  0.009         | Val Accuracy:  0.495         | Val f1:  0.495



100%|██████████| 1418/1418 [00:27<00:00, 50.92it/s]


Epochs: 7 | Train Loss:  0.009         | Train Accuracy:  0.493         | Train f1:  0.493         | Val Loss:  0.009         | Val Accuracy:  0.495         | Val f1:  0.495


In [42]:
#метрика дообученной модели на валидационном датасете
valid_f1 = F1Score().to(device)

total_acc_val = 0
for val_input, val_label in valid_loader:
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze().to(device)
    output = model(input_id, mask)
    valid_f1(output, val_label)
    acc = (output.argmax(dim=1) == val_label).sum().item()
    total_acc_val += acc
    
print(f'Val accuracy: {total_acc_val/len(valid_dataset):.3f}, F1 score: {valid_f1.compute().item():.3f}')

Val accuracy: 0.495, F1 score: 0.495


Дообученная модель показала увеличение метрик